## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-13-02-24-55 +0000,nyt,Trump’s D.C. Crackdown Plan Begins to Take Sha...,https://www.nytimes.com/2025/08/12/us/politics...
1,2025-08-13-02-20-29 +0000,bbc,AI start-up Perplexity makes $34.5bn bid for G...,https://www.bbc.com/news/articles/c3dpr0kkyz4o...
2,2025-08-13-02-14-13 +0000,startribune,Former Brooklyn Park council member wins DFL p...,https://www.startribune.com/voters-to-choose-d...
3,2025-08-13-02-13-27 +0000,nyt,D.C.’s Mayor Condemns Trump’s Deployment of th...,https://www.nytimes.com/live/2025/08/12/us/tru...
4,2025-08-13-02-11-47 +0000,latimes,"Rapper, YouTuber DDG briefly detained in SoCal...",https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,58
55,china,12
180,new,12
10,up,12
151,white,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
237,2025-08-12-10-35-02 +0000,nypost,JD Vance says Elon Musk should return to White...,https://nypost.com/2025/08/12/us-news/jd-vance...,104
12,2025-08-13-01-37-14 +0000,nypost,"Left-wing dark-money megadonors, including Geo...",https://nypost.com/2025/08/12/us-news/dark-mon...,104
124,2025-08-12-19-26-00 +0000,wsj,President Trump on Tuesday appeared to call fo...,https://www.wsj.com/economy/trump-calls-on-gol...,101
93,2025-08-12-21-08-00 +0000,wsj,The Trump administration is considering change...,https://www.wsj.com/politics/policy/trump-jobs...,100
146,2025-08-12-18-07-32 +0000,nypost,White House confirms Trump-Putin summit in Anc...,https://nypost.com/2025/08/12/us-news/white-ho...,99


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
237,104,2025-08-12-10-35-02 +0000,nypost,JD Vance says Elon Musk should return to White...,https://nypost.com/2025/08/12/us-news/jd-vance...
126,59,2025-08-12-19-21-00 +0000,wsj,The State Department released a long-delayed h...,https://www.wsj.com/politics/policy/state-depa...
65,57,2025-08-12-22-22-09 +0000,nypost,NYC ex-cons slice and dice their way through n...,https://nypost.com/2025/08/12/us-news/nyc-ex-c...
158,56,2025-08-12-17-33-00 +0000,wsj,Crypto Entrepreneur Do Kwon Pleads Guilty to F...,https://www.wsj.com/finance/currencies/crypto-...
10,46,2025-08-13-01-44-11 +0000,latimes,"She was killed while walking to her car, famil...",https://www.latimes.com/california/story/2025-...
12,46,2025-08-13-01-37-14 +0000,nypost,"Left-wing dark-money megadonors, including Geo...",https://nypost.com/2025/08/12/us-news/dark-mon...
224,43,2025-08-12-12-00-00 +0000,nypost,Ex-NYPD cop claims Bill de Blasio ‘didn’t like...,https://nypost.com/2025/08/12/us-news/ex-nypd-...
124,43,2025-08-12-19-26-00 +0000,wsj,President Trump on Tuesday appeared to call fo...,https://www.wsj.com/economy/trump-calls-on-gol...
169,39,2025-08-12-16-45-08 +0000,nyt,Philippines Condemns China After South China S...,https://www.nytimes.com/2025/08/12/world/asia/...
54,37,2025-08-12-22-53-38 +0000,nypost,What to expect from Friday’s Trump-Putin summi...,https://nypost.com/2025/08/12/world-news/what-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
